In [2]:
import osmnx as ox
import networkx as nx
import folium

In [45]:
# Get the road network for Bicol Region with additional highway types
place_name = ["Naga, Philippines", "Camarines Sur"]
custom_filter = '["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified"]'
graph = ox.graph_from_place(place_name, custom_filter=custom_filter)

# Coordinates of Barangay Halls (dispensing points)
dispensing_points = [
    {"province": "Camarines Sur", "City": "Naga", "barangay": "Cararayan", "lat": 13.628834653694073, "lon": 123.24006454110972},
    # Add more barangay hall locations here
]

# Creating new node for dispensing points doesn't work, so
# Just find the nearest nodes for each dispensing point and use it as the actual dp
for barangay in dispensing_points:
    nearest_node = ox.distance.nearest_nodes(graph, barangay['lon'], barangay['lat'])
    barangay['nearest_node'] = nearest_node

# Get the edges GeoDataFrame (2nd item in the tuple)
gdf_edges = ox.graph_to_gdfs(graph, nodes=False, edges=True)

# Get the centroid of Bicol Region to initialize the map
centroid = ox.geocode_to_gdf(place_name).unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=8, tiles='OpenStreetMap')

# Add the edges (roads) to the map
for _, edge in gdf_edges.iterrows():
    points = [(y, x) for x, y in edge['geometry'].coords]
    folium.PolyLine(points, color="darkred", weight=1.2, opacity=0.3).add_to(m)

# Add nearest node of Barangay Halls as nodes on the map
for barangay in dispensing_points:
    node = graph.nodes[barangay['nearest_node']]
    folium.Marker(location=[node['y'], node['x']],
                        radius=5, color='darkgreen', fill=True, fill_color='darkgreen', fill_opacity=1, icon=folium.Icon(color='darkgreen'),
                        popup=f"{barangay['barangay']}, {barangay['City']}, {barangay['province']}").add_to(m)

# Save the map
m.save("bicol_region_with_dispensing_points.html")